In [224]:
#Imports
import numpy as np
import pandas as pd
import plotly.express as px

In [225]:
#Sampling Data from CSV
df = pd.read_csv("Testing Data/3-3-2025/2025_03_04_00_19_40.0 (Kero 2).csv") #Change according to desired test

fire_start_index = df[df["MACHINE"] == 2].index.min()

sample_start_time = df.loc[fire_start_index-1, "PACKET_TIMESECONDS"]
sample_end_time = sample_start_time + 8

sample = df[(df["PACKET_TIMESECONDS"] >= sample_start_time) & (df["PACKET_TIMESECONDS"] <= sample_end_time)]

time_since_fire = sample["PACKET_TIMESECONDS"] - sample["PACKET_TIMESECONDS"].iloc[0]

sample["TIME_SINCE_FIRE"] = time_since_fire


C:\Users\goose\AppData\Local\Temp\ipykernel_33544\820575632.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [226]:
#Plotting
side = "KERO" #Change according to what you'd like ("LOX", "KERO", "FULL")

match side:
    case "LOX":
        pressure_list = ["NITROT", "LOXT", "LOXISO", "LOXMAN"]
    case "KERO":
        pressure_list = ["NITROT", "KEROT", "KEROISO", "KEROMAN"]
    case "FULL":
        pressure_list = ["NITROT", "LOXT", "LOXISO", "LOXMAN", "KEROT", "KEROISO", "KEROMAN"]

press_fig = px.line(sample, x = "TIME_SINCE_FIRE", y = pressure_list, title = "System Pressures vs Time")

press_fig.update_layout(
    xaxis_title="Time (seconds)",
    yaxis_title="Pressure (psi)",
    legend_title="Sensor",
)

flow_fig = px.line(sample, x = "TIME_SINCE_FIRE", y = "KEROFLOW", title = "System Flow Rates vs Time")

flow_fig.update_layout(
    xaxis_title="Time (seconds)",
    yaxis_title="Flow Rate (L/min)",
    legend_title="Sensor",
)

press_fig.show()
flow_fig.show()

In [227]:
#Averaging Measurements
averaging_sample = sample[(sample["TIME_SINCE_FIRE"] >= 2) & (sample["TIME_SINCE_FIRE"] <= 4)]

NITROT_ave = np.mean(averaging_sample["NITROT"])

LOXISO_ave = np.mean(averaging_sample["LOXISO"])
LOXT_ave = np.mean(averaging_sample["LOXT"])
LOXMAN_ave = np.mean(averaging_sample["LOXMAN"])

KEROISO_ave = np.mean(averaging_sample["KEROISO"])
KEROT_ave = np.mean(averaging_sample["KEROT"])
KEROMAN_ave = np.mean(averaging_sample["KEROMAN"])

FLOW_ave = np.mean(averaging_sample["KEROFLOW"])/(28.317*60) #convert L/min -> ft^3/s

print(f"Averaged NITROT: {NITROT_ave} \n")
print(f"Averaged LOXISO: {LOXISO_ave}")
print(f"Averaged LOXT: {LOXT_ave}")
print(f"Averaged LOXMAN: {LOXMAN_ave} \n")
print(f"Averaged KEROISO: {KEROISO_ave}")
print(f"Averaged KEROT: {KEROT_ave}")
print(f"Averaged KEROMAN: {KEROMAN_ave} \n")
print(f"Averaged FLOW: {FLOW_ave}")

Averaged NITROT: 919.1765548799996 

Averaged LOXISO: -33.011257322957206
Averaged LOXT: 4956.486201185046
Averaged LOXMAN: 4956.333087534194 

Averaged KEROISO: 496.7711009808537
Averaged KEROT: 475.4649661963619
Averaged KEROMAN: 301.25153783262743 

Averaged FLOW: 0.01611657237716845


In [228]:
#Setting dP's
dP_isolation_line_LOX = (LOXISO_ave - LOXT_ave)
dP_main_line_LOX = (LOXT_ave - LOXMAN_ave)
dP_injector_LOX = (LOXMAN_ave)

dP_isolation_line_KERO = (KEROISO_ave - KEROT_ave)
dP_main_line_KERO = (KEROT_ave - KEROMAN_ave)
dP_injector_KERO = (KEROMAN_ave)

#CdA Calculation

#Fluid Properties
rho_water = 1.940 #slugs/ft^3
rho_nitrogen_LOX = -2.05E-03 + 1.6E-04*(LOXISO_ave) + -5.3E-09*(LOXISO_ave)**2 #(2nd Order Polynomial fit to N2 Isothermal Density Data at T = 298K)
rho_nitrogen_KERO = -2.05E-03 + 1.6E-04*(KEROISO_ave) + -5.3E-09*(KEROISO_ave)**2 #(2nd Order Polynomial fit to N2 Isothermal Density Data at T = 298K)
gamma_nitrogen_LOX = 1.4 + 1.38E-04*(LOXISO_ave) + -1.47E-08*(LOXISO_ave)**2 #(2nd Order Polynomial fit to N2 Isothermal Gamma Data at T = 298K)
gamma_nitrogen_KERO = 1.4 + 1.38E-04*(KEROISO_ave) + -1.47E-08*(KEROISO_ave)**2 #(2nd Order Polynomial fit to N2 Isothermal Gamma Data at T = 298K)

#Mass flow of Nitrogen in LOX/Kero Lines
mass_flow_nitrogen_LOX = FLOW_ave * rho_nitrogen_LOX #slugs/s
mass_flow_nitrogen_KERO = FLOW_ave * rho_nitrogen_KERO #slugs/s

#Mass flow of Water in LOX/Kero Lines
mass_flow_water_LOX = FLOW_ave * rho_water #slugs/s
mass_flow_water_KERO = FLOW_ave * rho_water #slugs/s

#LOX Side CdA's
CdA_isolation_line_LOX = mass_flow_nitrogen_LOX/(np.sqrt(2*rho_nitrogen_LOX*dP_isolation_line_LOX*144)) #144 for psi -> psf
CdA_main_line_LOX = mass_flow_water_LOX/(np.sqrt(2*rho_water*dP_main_line_LOX*144)) #144 for psi -> psf
CdA_injector_LOX = mass_flow_water_LOX/(np.sqrt(2*rho_water*dP_injector_LOX*144)) #144 for psi -> psf
CdA_combined_LOX = mass_flow_water_LOX/(np.sqrt(2*rho_water*(dP_main_line_LOX+dP_injector_LOX)*144)) #combined main line + injector

#Kero Side CdA's
CdA_isolation_line_KERO = mass_flow_nitrogen_KERO/(np.sqrt(2*rho_nitrogen_KERO*dP_isolation_line_KERO*144)) #144 for psi -> psf
CdA_main_line_KERO = mass_flow_water_KERO/(np.sqrt(2*rho_water*dP_main_line_KERO*144)) #144 for psi -> psf
CdA_injector_KERO = mass_flow_water_KERO/(np.sqrt(2*rho_water*dP_injector_KERO*144)) #144 for psi -> psf
CdA_combined_KERO = mass_flow_water_KERO/(np.sqrt(2*rho_water*(dP_main_line_KERO+dP_injector_KERO)*144)) #combined main line + injector

#Unused Compressible CdA Relation
#CdA_isolation_line_LOX = mass_flow_nitrogen_LOX/(np.sqrt(2*rho_nitrogen_LOX*LOXISO_ave*144*(gamma_nitrogen_LOX/(gamma_nitrogen_LOX-1))*(((LOXT_ave/LOXISO_ave)**(2/gamma_nitrogen_LOX)) - ((LOXT_ave/LOXISO_ave)**((gamma_nitrogen_LOX+1)/gamma_nitrogen_LOX)))))

print(f"TRUST ONLY IF LINE WAS ACTUALLY PART OF TEST: \n")

print(f"CdA_isolation_line_LOX: {CdA_isolation_line_LOX}")
print(f"CdA_main_line_LOX: {CdA_main_line_LOX}")
print(f"CdA_injector_LOX: {CdA_injector_LOX}")
print(f"CdA_combined_LOX: {CdA_combined_LOX} \n")

print(f"CdA_isolation_line_KERO: {CdA_isolation_line_KERO}")
print(f"CdA_main_line_KERO: {CdA_main_line_KERO}")
print(f"CdA_injector_KERO: {CdA_injector_KERO}")
print(f"CdA_combined_KERO: {CdA_combined_KERO} \n")


TRUST ONLY IF LINE WAS ACTUALLY PART OF TEST: 

CdA_isolation_line_LOX: -1.1516602211579337e-06
CdA_main_line_LOX: 0.003380417551821849
CdA_injector_LOX: 1.8788714245949406e-05
CdA_combined_LOX: 1.8788424037247735e-05 

CdA_isolation_line_KERO: 5.6766091768716905e-05
CdA_main_line_KERO: 0.00010021586751615519
CdA_injector_KERO: 7.621012389762399e-05
CdA_combined_KERO: 6.066218411753553e-05 



In [229]:
#Expected Set Pressures
rho_LOX = 2.128531 #slugs/ft^3
rho_KERO = 1.59106 #slugs/ft^3

chamber_pressure = 565 #psi
mass_flow_total = 0.10148073 #slugs/s
OF_Ratio = 1.8

#Using propellant mass flows and CdA's to obtain tank pressures
mass_flow_LOX = mass_flow_total/((1/OF_Ratio)+1)
mass_flow_KERO = mass_flow_total/(OF_Ratio + 1)

expected_tank_press_LOX = ((mass_flow_LOX/CdA_combined_LOX)**2)/(2*rho_LOX*144) + chamber_pressure
expected_tank_press_KERO = ((mass_flow_KERO/CdA_combined_KERO)**2)/(2*rho_KERO*144) + chamber_pressure

#Using nitrogen mass flows and CdA's to obtain reg pressures
expected_rho_nitrogen_LOX = -2.05E-03 + 1.6E-04*(expected_tank_press_LOX) + -5.3E-09*(expected_tank_press_LOX)**2 #(2nd Order Polynomial fit to N2 Isothermal Density Data at T = 298K)
expected_rho_nitrogen_KERO = -2.05E-03 + 1.6E-04*(expected_tank_press_KERO) + -5.3E-09*(expected_tank_press_KERO)**2 #(2nd Order Polynomial fit to N2 Isothermal Density Data at T = 298K)
expected_gamma_nitrogen_LOX = 1.4 + 1.38E-04*(expected_tank_press_LOX) + -1.47E-08*(expected_tank_press_LOX)**2 #(2nd Order Polynomial fit to N2 Isothermal Gamma Data at T = 298K)
expected_gamma_nitrogen_KERO = 1.4 + 1.38E-04*(expected_tank_press_KERO) + -1.47E-08*(expected_tank_press_KERO)**2 #(2nd Order Polynomial fit to N2 Isothermal Gamma Data at T = 298K)

expected_mass_flow_nitrogen_LOX = (mass_flow_LOX*expected_rho_nitrogen_LOX)/(rho_LOX)
expected_mass_flow_nitrogen_KERO = (mass_flow_KERO*expected_rho_nitrogen_KERO)/(rho_KERO)

reg_press_LOX = ((expected_mass_flow_nitrogen_LOX/CdA_isolation_line_LOX)**2)/(2*expected_rho_nitrogen_LOX*144) + expected_tank_press_LOX
reg_press_KERO = ((expected_mass_flow_nitrogen_KERO/CdA_isolation_line_KERO)**2)/(2*expected_rho_nitrogen_KERO*144) + expected_tank_press_KERO

print(f"Set LOX Tank to: {expected_tank_press_LOX}")
print(f"Set KERO Tank to: {expected_tank_press_KERO} \n")

print(f"Set LOX Reg to: {reg_press_LOX}")
print(f"Set KERO Reg to: {reg_press_KERO} \n")


Set LOX Tank to: 20232.21832690681
Set KERO Tank to: 1343.9973344917416 

Set LOX Reg to: 2640734.8341434007
Set KERO Reg to: 1457.7319556893858 

